# Portfolio Optimization on the Jamaica Stock Exchange

##### Steps:
1. Scrape data
2. Clean data
3. Put data into a useful format
4. Add and calculate relevant metrics for risk and return
5. Store these data in MongoDB
6. Use this + user data-determined risk level to optimize portfolio with genetic algorithms (details coming soon)
7. Display results

### Importing libraries for web scraping and data cleaning

In [2]:
import requests # Get URL data
from bs4 import BeautifulSoup # Manipulate URL data
import datetime # useful since stock prices are IRL time-series data
from pandas import DataFrame as df # shortening to make easier
# for manipulating data after scraping
import numpy as np
import pandas as pd

### Steps 1-3: Scrape, clean and format data
We need a function to retrieve stock price data from the JSE website given any date. Let's get it to clean the data, format into useful data types and put it into a pandas dataframe.

In [3]:
def scrapePrices(date):
    url_string = "https://www.jamstockex.com/trading/trade-summary/?market=combined-market&date="
    date = str(date)[:10]
    test_page = requests.get(url_string + date)
    soup = BeautifulSoup(test_page.text, "html.parser")
    soup.prettify() # this gives HTML text for a given web page

    rows = soup.find_all("tr")
    tickers = []
    closingPrices = []
    for row in rows:
        rowData = row.get_text().split()
        if rowData[0] != "Symbol": 
            ticker = rowData[0]
            if ('USD' in rowData or "(USD)" in rowData) and "USD" not in rowData[0]:
                 ticker += "USD"
            tickers.append(ticker)
            try:
                price = rowData[-3]
                if ',' in price:
                    price = price.replace(',', '')
                price = float(price)
            except ValueError:
                price = rowData[-1]
                if ',' in price:
                    price = price.replace(',', '')
                price = float(price)
            closingPrices.append(price)  

    data = {"Ticker": tickers,
            date: np.array(closingPrices,dtype=object)}
    pdframe = df(data)
    return pdframe

Here's an example of this working:

In [288]:
scrapePrices(datetime.datetime(2021,2,4))

,Ticker,2021-02-04
0,AMG,1.62
1,BRG,13.89
2,BPOW,2.82
3,CHL,8.0
4,CABROKERS,1.85
...,...,...
74,PROVEN,37.0
75,SELECTF,0.61
76,SIL,3.0
77,TJH,1.33


### Running for more data
Time to scrape, clean and format much more data than for one day.

In [3]:
date = datetime.datetime(2021,2,1)
pdframe = scrapePrices(date)
pdframe = pdframe.set_index('Ticker')
for i in range(100):
    date += datetime.timedelta(days=1)
    frame = scrapePrices(date)
    frame = frame.set_index('Ticker')   
    pdframe = pdframe.merge(frame, on = 'Ticker', how = 'left')
pdframe = pdframe.drop_duplicates()
pdframe.to_csv('takealook.csv')
cleanData = pdframe.ffill(axis=1)
cleanData.to_csv('cleanData.csv')
cleanData

C:\Users\jason\anaconda3\lib\site-packages\pandas\core\reshape\merge.py:120: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  return op.get_result()


,2021-02-01,2021-02-02,2021-02-03,2021-02-04,2021-02-05,2021-02-06,2021-02-07,2021-02-08,2021-02-09,2021-02-10,...,2021-05-03,2021-05-04,2021-05-05,2021-05-06,2021-05-07,2021-05-08,2021-05-09,2021-05-10,2021-05-11,2021-05-12
Ticker,,,,,,,,,,,,,,,,,,,,,
AMG,1.61,1.60,1.60,1.62,1.62,1.62,1.62,1.66,1.66,1.63,...,1.79,1.79,1.79,1.79,1.79,1.79,1.79,1.74,1.74,1.74
BIL,82.29,82.32,82.36,81.49,81.06,81.06,81.06,81.57,80.01,79.35,...,84.94,84.91,84.96,87.60,86.56,86.56,86.56,85.84,86.19,86.09
CAC9.50,1.11,1.11,1.11,1.11,1.11,1.11,1.11,1.20,1.02,1.01,...,1.25,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00
CABROKERS,1.82,1.84,1.84,1.85,1.81,1.81,1.81,1.81,1.85,1.80,...,2.05,2.08,1.98,2.07,2.06,2.06,2.06,1.98,1.92,1.97
KREMI,5.05,5.04,4.95,5.00,5.00,5.00,5.00,4.95,4.95,4.97,...,5.94,6.10,5.91,6.00,5.86,5.86,5.86,5.34,6.05,5.98
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
LASD,3.18,3.15,3.35,3.25,3.37,3.37,3.37,3.18,3.26,3.23,...,3.94,3.86,4.22,3.86,4.00,4.00,4.00,4.18,4.30,4.35
PROVEN,37.00,37.00,37.00,37.00,37.04,37.04,37.04,38.73,38.19,37.36,...,38.00,38.31,38.22,37.59,38.60,38.60,38.60,37.55,38.61,38.26
SSLVC,0.53,0.54,0.55,0.55,0.55,0.55,0.55,0.55,0.55,0.55,...,0.69,0.65,0.58,0.63,0.64,0.64,0.64,0.64,0.64,0.64


In [4]:
cleanData = pd.read_csv("cleanData.csv").set_index("Ticker")
data = cleanData.transpose()
data

Ticker,AMG,BIL,CAC9.50,CABROKERS,KREMI,CFF,CPJ,CBNY,CWJDEFERREDA,PURITY,...,1834,DTL,FIRSTROCKJMD,HONBUN,JMMBGL7.50,LASD,PROVEN,SSLVC,TJH,WIG
2021-02-01,1.61,82.29,1.11,1.82,5.05,2.09,2.74,0.24,2.08,1.52,...,0.99,2.50,13.10,6.00,0.75,3.18,37.00,0.53,1.35,0.74
2021-02-02,1.60,82.32,1.11,1.84,5.04,1.95,2.71,0.22,1.90,1.56,...,0.99,2.49,12.93,5.87,0.69,3.15,37.00,0.54,1.35,0.74
2021-02-03,1.60,82.36,1.11,1.84,4.95,2.02,2.63,0.26,1.95,1.50,...,0.99,2.49,13.21,5.73,0.74,3.35,37.00,0.55,1.33,0.75
2021-02-04,1.62,81.49,1.11,1.85,5.00,2.00,2.63,0.26,2.08,1.41,...,0.99,2.50,13.35,5.82,0.75,3.25,37.00,0.55,1.33,0.75
2021-02-05,1.62,81.06,1.11,1.81,5.00,2.04,2.65,0.35,1.85,1.34,...,0.92,2.48,13.50,5.88,0.73,3.37,37.04,0.55,1.31,0.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-05-08,1.79,86.56,1.00,2.06,5.86,2.20,3.72,0.47,2.00,1.41,...,0.96,2.53,16.38,6.35,0.78,4.00,38.60,0.64,1.34,0.64
2021-05-09,1.79,86.56,1.00,2.06,5.86,2.20,3.72,0.47,2.00,1.41,...,0.96,2.53,16.38,6.35,0.78,4.00,38.60,0.64,1.34,0.64
2021-05-10,1.74,85.84,1.00,1.98,5.34,2.25,3.75,0.48,2.00,1.41,...,0.98,2.54,16.01,6.40,0.75,4.18,37.55,0.64,1.33,0.64
2021-05-11,1.74,86.19,1.00,1.92,6.05,2.35,3.75,0.47,2.00,1.65,...,0.96,2.59,16.38,6.40,0.75,4.30,38.61,0.64,1.33,0.66


### Markowitz Optimization
Now that we have the data, we use the PyPortfolioOpt library to find the portfolio weights under the standard Markowitz mean-variance optimization model. This gives us a benchmark for evaluating other approaches.

We'll start by picking a few stocks for our portfolio to give a smaller, more manageable example. Finding the % change for every day tells us the daily returns.

In [5]:
pick = pd.concat([data['BIL'],data['CPJ'],data['HONBUN'],data['CCC'],data['CBNY']],axis=1)
returns = pick.pct_change()
returns

,BIL,CPJ,HONBUN,CCC,CBNY
2021-02-01,NaN,NaN,NaN,NaN,NaN
2021-02-02,0.000365,-0.010949,-0.021667,-0.008335,-0.083333
2021-02-03,0.000486,-0.029520,-0.023850,0.008573,0.181818
2021-02-04,-0.010563,0.000000,0.015707,-0.021667,0.000000
2021-02-05,-0.005277,0.007605,0.010309,0.022147,0.346154
...,...,...,...,...,...
2021-05-08,0.000000,0.000000,0.000000,0.000000,0.000000
2021-05-09,0.000000,0.000000,0.000000,0.000000,0.000000
2021-05-10,-0.008318,0.008065,0.007874,-0.004457,0.021277
2021-05-11,0.004077,0.000000,0.000000,0.001119,-0.020833


Great. Some stocks move together in price and others move in opposite directions. A covariance matrix helps us see how they move, so let's get one ready.

In [5]:
cov_matrix = returns.cov() * 252
cov_matrix

,BIL,CPJ,HONBUN,CCC,CBNY
BIL,0.018136,-0.001378,0.002461,0.003387,0.024058
CPJ,-0.001378,0.336525,-0.001798,-0.035683,0.024008
HONBUN,0.002461,-0.001798,0.094534,-0.009734,0.008444
CCC,0.003387,-0.035683,-0.009734,0.224427,-0.025280
CBNY,0.024058,0.024008,0.008444,-0.025280,5.106977


Installing and importing dependencies since I didn't do that yet.

In [400]:
pip install PyPortfolioOpt

Note: you may need to restart the kernel to use updated packages.


In [6]:
from pypfopt import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns

Optimizing with the library for return and risk.

In [7]:
mu = expected_returns.mean_historical_return(data)
S = risk_models.sample_cov(data)
ef = EfficientFrontier(mu,S)
weights = ef.max_sharpe()
cleaned_weights = ef.clean_weights()
print(cleaned_weights)
ef.portfolio_performance(verbose=True)

C:\Users\jason\anaconda3\lib\site-packages\pypfopt\risk_models.py:68: UserWarning: The covariance matrix is non positive semidefinite. Amending eigenvalues.
  warnings.warn(


OrderedDict([('AMG', 0.0), ('BIL', 0.0), ('CAC9.50', 0.00152), ('CABROKERS', 0.0), ('KREMI', 0.0), ('CFF', 0.0), ('CPJ', 0.01934), ('CBNY', 0.00442), ('CWJDEFERREDA', 0.0), ('PURITY', 0.00427), ('ELITE', 0.0), ('CPFV', 0.00568), ('FIRSTROCKUSD', 0.0), ('GENAC', 0.01788), ('GK', 0.06846), ('ICREATE', 0.0085), ('JBG', 0.0), ('JAMT', 0.00994), ('JETCON', 0.0), ('JMMBGL', 0.02557), ('KW', 0.0), ('LASM', 0.0473), ('LUMBER', 0.0), ('MAILPAC', 0.0), ('MDS', 0.0), ('MPCCEL', 0.00521), ('NCBFG', 0.0), ('QWI', 0.00895), ('RJR', 0.0), ('SELECTF', 0.05195), ('SALF', 0.0), ('SGJ', 0.01504), ('SOS', 0.0206), ('SVL', 0.0), ('SCIJMDUSD', 0.0), ('SCIUSD', 0.01936), ('LAB', 0.0199), ('TROPICAL', 0.02171), ('TTECH', 0.0051), ('WISYNCO', 0.0), ('138SL', 0.0), ('AFS', 0.02053), ('BRG', 0.0), ('CHL', 0.00268), ('CCC', 0.0), ('CAR', 0.0527), ('DCOVE', 0.0), ('EPLY', 0.03859), ('ECL', 0.0), ('FTNA', 0.06393), ('FOSRICH', 0.03729), ('INDIES', 0.04918), ('JP', 0.01887), ('JSE', 0.0), ('KLE', 0.0), ('KEY', 0.027

(0.7587766586379452, 0.04367502025218471, 16.915313476036456)

Now let's try to see if it will work for all our data, and if so, what will happen.

In [8]:
mu = expected_returns.mean_historical_return(data)
S = risk_models.sample_cov(data)
ef = EfficientFrontier(mu,S)
weights = ef.max_sharpe()
cleaned_weights = ef.clean_weights()
print(cleaned_weights)
ef.portfolio_performance(verbose=True)

OrderedDict([('AMG', 0.0), ('BIL', 0.0), ('CAC9.50', 0.00152), ('CABROKERS', 0.0), ('KREMI', 0.0), ('CFF', 0.0), ('CPJ', 0.01934), ('CBNY', 0.00442), ('CWJDEFERREDA', 0.0), ('PURITY', 0.00427), ('ELITE', 0.0), ('CPFV', 0.00568), ('FIRSTROCKUSD', 0.0), ('GENAC', 0.01788), ('GK', 0.06846), ('ICREATE', 0.0085), ('JBG', 0.0), ('JAMT', 0.00994), ('JETCON', 0.0), ('JMMBGL', 0.02557), ('KW', 0.0), ('LASM', 0.0473), ('LUMBER', 0.0), ('MAILPAC', 0.0), ('MDS', 0.0), ('MPCCEL', 0.00521), ('NCBFG', 0.0), ('QWI', 0.00895), ('RJR', 0.0), ('SELECTF', 0.05195), ('SALF', 0.0), ('SGJ', 0.01504), ('SOS', 0.0206), ('SVL', 0.0), ('SCIJMDUSD', 0.0), ('SCIUSD', 0.01936), ('LAB', 0.0199), ('TROPICAL', 0.02171), ('TTECH', 0.0051), ('WISYNCO', 0.0), ('138SL', 0.0), ('AFS', 0.02053), ('BRG', 0.0), ('CHL', 0.00268), ('CCC', 0.0), ('CAR', 0.0527), ('DCOVE', 0.0), ('EPLY', 0.03859), ('ECL', 0.0), ('FTNA', 0.06393), ('FOSRICH', 0.03729), ('INDIES', 0.04918), ('JP', 0.01887), ('JSE', 0.0), ('KLE', 0.0), ('KEY', 0.027

(0.7587766586379452, 0.04367502025218471, 16.915313476036456)

So taking the whole market into account somehow gives a lower expected annual return, but for WAY less "risk". Below we simply sort the portfolio to show the heaviest-weighted stocks first for ease of reference.

In [13]:
portfolio = [('AMG', 0.0), ('BIL', 0.0), ('CAC9.50', 0.00152), ('CABROKERS', 0.0), ('KREMI', 0.0), ('CFF', 0.0), ('CPJ', 0.01934), ('CBNY', 0.00442), ('CWJDEFERREDA', 0.0), ('PURITY', 0.00427), ('ELITE', 0.0), ('CPFV', 0.00568), ('FIRSTROCKUSD', 0.0), ('GENAC', 0.01788), ('GK', 0.06846), ('ICREATE', 0.0085), ('JBG', 0.0), ('JAMT', 0.00994), ('JETCON', 0.0), ('JMMBGL', 0.02557), ('KW', 0.0), ('LASM', 0.0473), ('LUMBER', 0.0), ('MAILPAC', 0.0), ('MDS', 0.0), ('MPCCEL', 0.00521), ('NCBFG', 0.0), ('QWI', 0.00895), ('RJR', 0.0), ('SELECTF', 0.05195), ('SALF', 0.0), ('SGJ', 0.01504), ('SOS', 0.0206), ('SVL', 0.0), ('SCIJMDUSD', 0.0), ('SCIUSD', 0.01936), ('LAB', 0.0199), ('TROPICAL', 0.02171), ('TTECH', 0.0051), ('WISYNCO', 0.0), ('138SL', 0.0), ('AFS', 0.02053), ('BRG', 0.0), ('CHL', 0.00268), ('CCC', 0.0), ('CAR', 0.0527), ('DCOVE', 0.0), ('EPLY', 0.03859), ('ECL', 0.0), ('FTNA', 0.06393), ('FOSRICH', 0.03729), ('INDIES', 0.04918), ('JP', 0.01887), ('JSE', 0.0), ('KLE', 0.0), ('KEY', 0.02772), ('KPREIT', 0.0), ('KEX', 0.01893), ('LASF', 0.0), ('MIL', 0.00506), ('MJE', 0.01321), ('PAL', 0.0), ('PJAM', 0.0), ('PTL', 0.0), ('PROVENUSD', 0.0), ('PULS', 0.0114), ('SJ', 0.00312), ('XFUND', 0.0), ('SELECTMD', 0.02143), ('SEP', 0.02029), ('SIL', 0.0), ('SCIJMD', 0.0), ('TJHUSD', 0.0), ('VMIL', 0.0), ('1834', 0.0), ('DTL', 0.11183), ('FIRSTROCKJMD', 0.04241), ('HONBUN', 0.03172), ('JMMBGL7.50', 0.0), ('LASD', 0.0), ('PROVEN', 0.0), ('SSLVC', 0.00836), ('TJH', 0.02003), ('WIG', 0.0)]
portfolio.sort(key=lambda x: x[1])
portfolio.reverse()
portfolio

[('DTL', 0.11183),
 ('GK', 0.06846),
 ('FTNA', 0.06393),
 ('CAR', 0.0527),
 ('SELECTF', 0.05195),
 ('INDIES', 0.04918),
 ('LASM', 0.0473),
 ('FIRSTROCKJMD', 0.04241),
 ('EPLY', 0.03859),
 ('FOSRICH', 0.03729),
 ('HONBUN', 0.03172),
 ('KEY', 0.02772),
 ('JMMBGL', 0.02557),
 ('TROPICAL', 0.02171),
 ('SELECTMD', 0.02143),
 ('SOS', 0.0206),
 ('AFS', 0.02053),
 ('SEP', 0.02029),
 ('TJH', 0.02003),
 ('LAB', 0.0199),
 ('SCIUSD', 0.01936),
 ('CPJ', 0.01934),
 ('KEX', 0.01893),
 ('JP', 0.01887),
 ('GENAC', 0.01788),
 ('SGJ', 0.01504),
 ('MJE', 0.01321),
 ('PULS', 0.0114),
 ('JAMT', 0.00994),
 ('QWI', 0.00895),
 ('ICREATE', 0.0085),
 ('SSLVC', 0.00836),
 ('CPFV', 0.00568),
 ('MPCCEL', 0.00521),
 ('TTECH', 0.0051),
 ('MIL', 0.00506),
 ('CBNY', 0.00442),
 ('PURITY', 0.00427),
 ('SJ', 0.00312),
 ('CHL', 0.00268),
 ('CAC9.50', 0.00152),
 ('WIG', 0.0),
 ('PROVEN', 0.0),
 ('LASD', 0.0),
 ('JMMBGL7.50', 0.0),
 ('1834', 0.0),
 ('VMIL', 0.0),
 ('TJHUSD', 0.0),
 ('SCIJMD', 0.0),
 ('SIL', 0.0),
 ('XFUND', 

### Writing our own portfolio evaluation function to compare to benchmark
NB: PyPortfolioOpt has some whack way of calculating expected returns since it annualizes them, so we're gonna use this function to evaluate whatever portfolios we get.

In [10]:
data.iloc[-30]

Ticker
AMG           1.82
BIL          86.50
CAC9.50       1.47
CABROKERS     2.11
KREMI         6.00
             ...  
LASD          3.78
PROVEN       35.18
SSLVC         0.63
TJH           1.37
WIG           0.63
Name: 2021-04-13, Length: 84, dtype: float64

In [14]:
portfolioValues = [] # @MARLON THIS LINE IS VERY IMPORTANT
def evalPortfolio(portfolio,data,policyRate=0.02):
    # Accepts a list of 2-tuples: ('ticker',weight between 0 and 1) and returns expected annual
    # return and annual volatility. May implement with dictionary
    portfolioReturn = 0
    returns = (data.iloc[-1] - data.iloc[0])/data.iloc[0]
    for asset in portfolio:
        ticker, weight = asset
        portfolioReturn += returns[ticker] * weight
        
    
    for i in range(len(data)):
        dailyPortfolioValue = sum([x[1] * data.iloc[i][x[0]] for x in portfolio])
        portfolioValues.append(dailyPortfolioValue)
        
    #print('Return: ' + '{:.1%}'.format(portfolioReturn))
  
    S = risk_models.sample_cov(data)
    portfolio.sort(key=lambda x: list(data.columns).index(x[0]))
    weights = np.array([x[1]for x in portfolio])
    variance = np.dot(weights.T,np.dot(S,weights))
    volatility = np.sqrt(variance)  
    #print('Volatility: ' + '{:.1%}'.format(volatility))
    sharpeRatio = (portfolioReturn - policyRate)/volatility
    #print("Sharpe Ratio: " + '{:.3}'.format(sharpeRatio))
    return (portfolioReturn,volatility,sharpeRatio)    

In [17]:
tempTest = evalPortfolio(portfolio,data)

### Track portfolio
This is just 2 lines of code really

In [44]:
datesAndValues = list(zip(list(data.index.values),portfolioValues))
datesAndValues

[('2021-02-01', 13.774150199999998),
 ('2021-02-02', 14.010940199999995),
 ('2021-02-03', 14.175485699999994),
 ('2021-02-04', 14.038586199999997),
 ('2021-02-05', 13.975393100000002),
 ('2021-02-06', 13.975393100000002),
 ('2021-02-07', 13.975393100000002),
 ('2021-02-08', 14.118743399999996),
 ('2021-02-09', 14.087012699999999),
 ('2021-02-10', 14.4028365),
 ('2021-02-11', 14.472916699999999),
 ('2021-02-12', 14.530268999999999),
 ('2021-02-13', 14.530268999999999),
 ('2021-02-14', 14.530268999999999),
 ('2021-02-15', 14.5534061),
 ('2021-02-16', 14.199816699999998),
 ('2021-02-17', 14.199816699999998),
 ('2021-02-18', 14.5003022),
 ('2021-02-19', 14.458896199999998),
 ('2021-02-20', 14.458896199999998),
 ('2021-02-21', 14.458896199999998),
 ('2021-02-22', 14.467463600000002),
 ('2021-02-23', 14.6642938),
 ('2021-02-24', 14.676464600000003),
 ('2021-02-25', 14.8258116),
 ('2021-02-26', 14.829518),
 ('2021-02-27', 14.829518),
 ('2021-02-28', 14.829518),
 ('2021-03-01', 15.380822799999

# Experiment #1: Genetic algorithms
Phew, so that groundwork is out the way. It's now time to try a genetic algorithm to find the globally best portfolio. 

We'll accept several parameters like age, net worth and annual income as well as self-reported risk to fine-tune a portfolio for a given person, since the best portfolio for one person might be totally different from another (let's just try not to lose money :)).

Let's start by writing formulae that determine some key metrics.

### Risk levels
Older investors who are more risk averse, more dependent on their salaries and hold more concentrated portfolios carry a higher risk level.

Younger investors who are more risk tolerant, are less dependent on their salaries and hold more diversified portfolios carry a lower risk level.

In this framework, the lowest possible risk adjustment value is 0.65 and the highest is 1.475

In [9]:
def age_risk(age):
    # Returns a lower risk for younger persons since they have more time to recover from losses
    if age < 100:
        return age/100
    else:
        return 1

# Income to net worth ratio
def nw_income(net_worth,salary):
    # Returns a lower risk for those who rely on their salary less (ASSUMPTION)
    return salary/net_worth

# Self-reported risk
def reported_risk(risk):
    # Risk is a number between 0 and 10
    # Returns a lower risk aversion to persons who have a higher risk appetite
    return (10-risk)/10

def concentration_risk(portfolio):
    # Applying a modest concentration risk metric based on the Herfindahl–Hirschman index
    return sum([(x[1])**2 for x in portfolio])
    
def risk_premium(portfolio,age,net_worth,salary,reportedRisk):
    # Averages all the above functions to one risk level
    # 0.5 added to normalize output range
    return np.mean(age_risk(age) + nw_income(net_worth,salary) \
+ reported_risk(reportedRisk)) + 0.5
   

### Transaction costs 
Each transaction in the portfolio incurs a cost, and it would be unrealistic to ignore this fact. Values are calculated at current rates in Jamaica.

In [12]:
def transaction_cost(transaction_value,broker_commission=0.005):
    commission_fee = broker_commission * transaction_value
    cess = 0.003 * transaction_value
    trading_fee = 0.003 * transaction_value
    gct = 0.15 * (commission_fee + cess + trading_fee)
    return sum(commission_fee,cess,trading_fee,gct)

### Expected portfolio change
We invest to get richer. This does not seek to predict the future value of any stocks, but rather see if by rebalancing our portfolio in a certain way, we would increase our portfolio value.

In [13]:
def find_transaction_value(current_portfolio,new_portfolio,portfolio_value):
    # helper function to give a dollar value to the stocks that we move
    value = 0
    for i in len(current_portfolio):
        change = abs(current_portfolio[i][1] - new_portfolio[i][1])
        value += change * portfolio_value
    return value    

def expected_portfolio_value_change(current_portfolio,new_portfolio,data,portfolio_value):
    returns = data.pct_change()
    current_expected_value = np.sum(returns.mean() * [x[1] for x in current_portfolio] * 252)
    new_expected_value = np.sum(returns.mean() * [x[1] for x in new_portfolio] * 252)
    # We have two portfolios. We have to trade assets to move from one to the other.
    # We need to find the value of these transactions (trades) to then substract the costs associated.
    transaction_value = find_transaction_value(current_portfolio, new_portfolio, portfolio_value)
    transaction_fees = transaction_cost(transaction_value)
    net_change = new_expected_value - current_expected_value - transaction_fees
    return net_change        

#### Fitness function
Ok so now the metrics to assess a move are down. We have reason to move ahead (prospective net returns/ expected portfolio value change) and reason to be cautious (risk). Now let's put those together to determine how fit a portfolio is.

In [39]:
def fitness(portfolio,age,net_worth,salary,reportedRisk,policy_rate=0.02):
    # Simple since we abstracted almost everything away
    # The higher the output, the better
    returns, volatility, sharpe = evalPortfolio(portfolio,data,policy_rate)
    premium = risk_premium(portfolio,age,net_worth,salary,reportedRisk)
    risk_rating = volatility * ((0.9 * premium) + (0.1 * concentration_risk(portfolio)))
    return returns/risk_rating    

#### Reproduction and mutation functions

In [70]:
import random
#import itertools
def initialize(data, n=48):
    # We create n portfolios with random weights as our initial population
    population = []
    for _ in range(n):
        population.append(list(zip(list(data.columns),np.random.dirichlet(np.ones(len(data.columns))\
                                                           ,size=1).tolist()[0])))
    return population

# def initialize(data, n=48):
#     # We create n portfolios with random weights as our initial population
#     population = []
#     for _ in range((3*n)//4):
#         population.append(list(zip(list(data.columns),np.random.dirichlet(np.ones(len(data.columns))\
#                                                            ,size=1).tolist()[0])))
#     for _ in range(n//4):
#         index = random.randint(0,len(data.columns))
#         lst = list(itertools.repeat(1,len(data.columns)-1))
#         lst[index] = 0.5
#         lst[index-1] = 0.3
#         lst[index+1] = 0.2
#         population.append(list(zip(list(data.columns),lst)))
#     return population

# Some values I need for testing
age = 21
net_worth = 500000
salary = 100000
reportedRisk = 5
    
def select_chromosome(population):
    # Returns the best portfolio of the population
    # if half, returns the fitter half of the population
    fitness_values = list(zip(range(len(population)),\
                             [fitness(portfolio,age,net_worth,salary,reportedRisk) \
                              for portfolio in population]))
  
#     if half:
#         fitness_values.sort(key=lambda x: x[1])
#         indices = list(filter(lambda x: x[0],fitness_values[(len(population)//2):]))
#         return [population[x] for x,y in indices]   
    
    # We have to normalize the values now
    #normalized_fitness_values = [raw_fitness/sum(fitness_values) for raw_fitness in fitness_values]
    #print(normalized_fitness_values)   

    return (population[fitness_values.index(max(fitness_values))], max(fitness_values)[1])
    
#     More traditional approach that I happen to think is less appropriate here    
#     accumulated_fitness_values = list(np.cumsum(normalized_fitness_values))
#     selector = random.random()
#     return population[list(map(lambda x: x >= selector, accumulated_fitness_values)).index(True)]
    
def crossover(population):
    def normalize(portfolio):
        weights_sum = sum(map(lambda x: x[1],portfolio))
        return [(ticker,weight/weights_sum) for ticker,weight in portfolio]
    alpha = 0.5 # probability of parent 1's genes passing on rather than parent 2's
    cross = [0,1]   
    new_generation = []
    mating_pool = random.sample(population,len(population)//2)
    
    best_chromosome, fitness_val = select_chromosome(population)
    print("Fitness on this iteration")
    print(fitness_val)
    best_chromosome.sort()
    for portfolio in mating_pool:
        portfolio.sort()
        use_best = random.choices(cross,cum_weights=[70,100])[0]
        mutation = random.choices(cross,cum_weights=[90,100])[0]
        child1 = []
        child2 = []
        if use_best:
            for asset in range(len(portfolio)):
                child1.append((portfolio[asset][0],\
                               alpha * portfolio[asset][1] + \
                               (1 - alpha) * best_chromosome[asset][1]))
                
                child2.append((portfolio[asset][0],\
                               (1 - alpha) * portfolio[asset][1] + \
                               alpha * best_chromosome[asset][1]))
        else:
            parent1 = population[random.randint(0,len(population)-1)]
            parent1.sort()
            for asset in range(len(portfolio)):
                child1.append((portfolio[asset][0],\
                               alpha * portfolio[asset][1] + \
                               (1 - alpha) * parent1[asset][1]))
                child2.append((portfolio[asset][0],\
                               (1 - alpha) * portfolio[asset][1] + \
                               alpha * parent1[asset][1]))
        child1 = normalize(child1)
        child2 = normalize(child2)        
        if mutation:           
            child1 = mutate(child1)
            child2 = mutate(child2)
        new_generation.append(child1)
        new_generation.append(child2)  
    return (new_generation,fitness_val)         
                                        
def mutate(portfolio):
    # We swap 2 random stocks' weights
    asset1position = random.randint(0,len(portfolio)-1)
    asset2position = random.randint(0,len(portfolio)-1)
    portfolio[asset1position], portfolio[asset2position] = portfolio[asset2position], portfolio[asset1position]
    return portfolio

### Actually doing the experiment


In [34]:
select_chromosome(initialize(data))

MAX
0.901760491806537


([('AMG', 0.0161156699943273),
  ('BIL', 0.005053505603701022),
  ('CAC9.50', 0.0031297244012710684),
  ('CABROKERS', 0.0034276337340716024),
  ('KREMI', 0.00591820385627274),
  ('CFF', 0.0027394252057345264),
  ('CPJ', 0.007751676529662146),
  ('CBNY', 0.017326300181239224),
  ('CWJDEFERREDA', 0.013461154461220088),
  ('PURITY', 0.0022896143482380262),
  ('ELITE', 0.010291263036501049),
  ('CPFV', 0.022748175342884477),
  ('FIRSTROCKUSD', 0.0008128281172508796),
  ('GENAC', 0.004094758209844551),
  ('GK', 0.0010192193125209054),
  ('ICREATE', 0.04850379965775115),
  ('JBG', 0.0010517243454545285),
  ('JAMT', 0.014806853057865938),
  ('JETCON', 0.0009505807939739917),
  ('JMMBGL', 0.009652038180794985),
  ('KW', 0.015085918071900132),
  ('LASM', 0.006316021848398584),
  ('LUMBER', 0.007710663677646238),
  ('MAILPAC', 0.01602509635725607),
  ('MDS', 0.060996929643585496),
  ('MPCCEL', 0.008516993403056707),
  ('NCBFG', 0.008945834014201261),
  ('QWI', 0.003501702063980861),
  ('RJR', 0.

In [71]:
# Initialize population
population = initialize(data)
print("Population initialized")
# Crossover for 16 generations
generation_count = 0
max_generations = 16
fitness_list = []
best = [[],0] # helps to save the best population encountered in case of high local maxima
while (generation_count < max_generations):
    print("Generation: " + str(generation_count))
    population,current_fitness = crossover(population)
    if current_fitness > best[1]: # saving best so far
        best[0], best[1] = population, current_fitness    
    
    print("Population size: " + str(len(population)) )
    fitness_list.append(current_fitness)
    if len(fitness_list) > 3: # avoids getting stuck on local maxima
        if abs(fitness_list[-1] - fitness_list[-2]) < 0.02 and\
        abs(fitness_list[-2] - fitness_list[-3]) < 0.02:
            print("Fitness converged. Stopping iteration.")
            break
    generation_count += 1
new_portfolio, _ = select_chromosome(best[0])

Population initialized
Generation: 0
Fitness on this iteration
1.3060329577390681
Population size: 48
Generation: 1
Fitness on this iteration
1.2862019008715226
Population size: 48
Generation: 2
Fitness on this iteration
1.2862019008715226
Population size: 48
Generation: 3
Fitness on this iteration
1.2178321465402293
Population size: 48
Generation: 4
Fitness on this iteration
1.188862431653772
Population size: 48
Generation: 5
Fitness on this iteration
1.20583995048288
Population size: 48
Generation: 6
Fitness on this iteration
1.1999395110253979
Population size: 48
Fitness converged. Stopping iteration.


In [72]:
new_portfolio.sort(key=lambda x: x[1])
new_portfolio.reverse()
print(new_portfolio)
evalPortfolio(new_portfolio,data)

[('PROVEN', 0.03936131313332757), ('CPJ', 0.038670698956695775), ('KEX', 0.034359905455374457), ('ELITE', 0.03277072272078056), ('NCBFG', 0.029095495485092635), ('SVL', 0.028849512522637722), ('LASD', 0.028771265962730715), ('MAILPAC', 0.02621173731447473), ('BRG', 0.025610376165897335), ('CHL', 0.02528239198688499), ('PURITY', 0.023572178123203606), ('JP', 0.02281601629706327), ('FOSRICH', 0.022641778009682128), ('SGJ', 0.0222500766866918), ('KW', 0.021752793013013538), ('GK', 0.021451767643728484), ('LASF', 0.017598085651804027), ('JMMBGL7.50', 0.016238473289983863), ('BIL', 0.01611493849000657), ('JAMT', 0.015795860287773703), ('TJHUSD', 0.014926406864167371), ('138SL', 0.014383694543584761), ('CWJDEFERREDA', 0.013989511742828455), ('MJE', 0.013923658774824632), ('ICREATE', 0.013875005145554873), ('SELECTMD', 0.01386750847007138), ('SCIJMDUSD', 0.013847821081806006), ('JETCON', 0.013249721559068084), ('SELECTF', 0.012883657753532713), ('FTNA', 0.012806492585468256), ('CCC', 0.012476

(0.14863133810520393, 0.09092893676149809, 1.4146358979495968)

In [ ]:
# Apache spark
# priority queues/trees/other data structures
# time complexity
# minimum feature set properly first
# try particle swarm

# Midterm: Project summary, different approaches tried, challenges and how overcome

In [45]:
portfolio = [('PAL', 0.05066169379125782), ('XFUND', 0.07978949058495982), ('AMG', 0.518934177124895743), ('SOS', 0.01846314492726898), ('LASM', 0.018206047773534406), ('PJAM', 0.018066710231708662), ('FIRSTROCKJMD', 0.01719958866225623), ('ECL', 0.016772055330670107), ('WIG', 0.016762847567984168), ('JMMBGL', 0.01631258176294159), ('GENAC', 0.016182301074646133), ('INDIES', 0.0152298634177669), ('SJ', 0.014886336558112433), ('MDS', 0.014730577848164585), ('GK', 0.014577203199825584), ('CHL', 0.014544095533142107), ('SELECTF', 0.01446000568970937), ('SCIUSD', 0.014267578797331983), ('TTECH', 0.013789062036041631), ('NCBFG', 0.013692376291780203), ('JBG', 0.013592935804390227), ('FTNA', 0.013509172594945537), ('LASF', 0.013282841826407307), ('LUMBER', 0.013202931183171661), ('CPJ', 0.013176318817053284), ('1834', 0.013148138497843504), ('EPLY', 0.012767475359549087), ('MIL', 0.012753713319621263), ('KEY', 0.012696261346557953), ('TJH', 0.012470392212054168), ('PTL', 0.01232702571372029), ('MPCCEL', 0.01230469853689464), ('KLE', 0.012135320242346849), ('CAC9.50', 0.012077239442042446), ('CAR', 0.012053974789213787), ('JSE', 0.012042304940635313), ('FOSRICH', 0.012013925039343875), ('BIL', 0.011736848473235398), ('WISYNCO', 0.011707440210217229), ('MAILPAC', 0.011632397260302395), ('CFF', 0.011612727774922518), ('DTL', 0.011516503648001544), ('FIRSTROCKUSD', 0.011512646504292378), ('SALF', 0.011183230300026966), ('ICREATE', 0.011171027610628907), ('TJHUSD', 0.011098268519338246), ('138SL', 0.011052494844416587), ('CCC', 0.01094361700763062), ('SEP', 0.010895437210595509), ('SGJ', 0.010791499223000997), ('SELECTMD', 0.010772292508107887), ('CBNY', 0.010576443918440115), ('JMMBGL7.50', 0.010571601886609706), ('QWI', 0.010561841912050103), ('DCOVE', 0.01049416331074614), ('SIL', 0.01039833900309583), ('SCIJMD', 0.010381327113247978), ('CABROKERS', 0.01037483827659858), ('LASD', 0.010360374626383421), ('KPREIT', 0.01034717593821227), ('CPFV', 0.010202775897805258), ('KEX', 0.010124433961914278), ('SCIJMDUSD', 0.010026524484264094), ('TROPICAL', 0.010008223681455151), ('BRG', 0.009928722813183128), ('KW', 0.009711417059962911), ('VMIL', 0.009521725862809602), ('MJE', 0.009486474362579688), ('CWJDEFERREDA', 0.009416141826517224), ('JP', 0.009396220375032531), ('JAMT', 0.009310480871640854), ('PULS', 0.009142054328926804), ('LAB', 0.008451236561998232), ('JETCON', 0.008371364321771421), ('HONBUN', 0.008028564533991837), ('PROVENUSD', 0.007986408576188186), ('ELITE', 0.007925523190957044), ('PURITY', 0.007880800139679203), ('PROVEN', 0.007650909440694333), ('RJR', 0.007146102332568146), ('KREMI', 0.00667289729715838), ('SVL', 0.00661960789613431), ('AFS', 0.006137309586152674), ('SSLVC', 0.006079135646723741)]

In [52]:
def normalize(portfolio):
        weights_sum = sum(map(lambda x: x[1],portfolio))
        print(weights_sum)
        return [(ticker,weight/weights_sum) for ticker,weight in portfolio]

In [ ]:
def normalize2(portfolio):
    

1.0000000000000004

# Extras that don't seem to work (yet) so ignore

In [383]:
pBar = logReturns.mean()
Sigma = logReturns.cov()

In [306]:
from scipy.optimize import minimize

In [384]:
rMin = 0.02
def riskFunction(w):
    return np.dot(w.T, np.dot(Sigma,w))
init_weight = 0.25 # round(1/len(data.columns),10)
w0 = []
bounds = []
for _ in range(4):
    w0.append(init_weight)
    bounds.append((0,1))
    
def checkMinReturn(w):
    return rMin - np.sum(pBar*w)

def checkSumToOne(w):
    return np.sum(w)-1

constraints = ({'type':'eq', 'fun':checkMinReturn},{'type':'eq', 'fun':checkSumToOne})
w_opt = minimize(riskFunction,w0,method='SLSQP',bounds=bounds,constraints=constraints)

In [389]:
def markowitz(rMin,Sigma,pBar):
    N = len(Sigma)
    o = np.ones(N)
    SigmaInv = np.linalg.inv(Sigma)
    a = np.dot(pBar.T,np.dot(SigmaInv,pBar))
    b = np.dot(pBar.T,np.dot(SigmaInv,o))
    c = np.dot(o.T,np.dot(SigmaInv,o))
    return (1/(a*c - b**2)) * np.dot(SigmaInv,((c*rMin - b) * pBar + (a-b*rMin)*o))

In [390]:
w_opt = markowitz(rMin,Sigma,pBar)
w_opt

array([-4.67193548,  2.39761593, -0.69221169,  3.96653124])

In [378]:
riskFunction(w_opt)

0.019562867561516338